In [1]:
import pygame
import sys
from pygame.locals import *
from math import sin, cos, tan, pi
import numpy as np
from numpy.linalg import inv
from spring import spring

pygame 2.4.0 (SDL 2.26.4, Python 3.9.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


__Spring Pendulum Simulation__

Numerical methods are not only necessary to solve any equation that cant be solved analytically but in code its impossible to do anything but a numerical solution. Here we use RK4 to numerically solve the spring pendulum EOM to be able to simulate it in a cool anamation.

Feel free to mess with the variables and inital conditions and see how the simulation changes!

When the simulation is running press T to turn the path tracing on and off, press C to reset the tracing, and press Q to quit the simulation. The kernel may crash after quitting the program so press the "restart" button at the top to reset the Kernel to run the program again. Enjoy!

**Setting up Variables and Initial Conditions** 

In [2]:
# variables
m = 6.0 # kg
l0 = 3.5 # m
g = 9.81 # m/s^2
k = 100.0 # N/m

prev_point = None
t = 0.0 # s
delta_t = 0.02 # s

# ICS [dx, dtheta, x, theta]
y = np.array([0.0, 0.0*np.pi, 0.0, 0.5*np.pi])

**Simulation set up and RK4**

In [3]:
class Spring():
	def __init__(self, color, start, end, nodes, width, lead1, lead2):
		self.start = start
		self.end = end
		self.nodes = nodes
		self.width = width
		self.lead1 = lead1
		self.lead2 = lead2
		self.weight = 3
		self.color = color

	def update(self, start, end):
		self.start = start
		self.end = end

		self.x, self.y, self.p1, self.p2 = spring(self.start, self.end, self.nodes, self.width, self.lead1, self.lead2)
		self.p1 = (int(self.p1[0]), int(self.p1[1]))
		self.p2 = (int(self.p2[0]), int(self.p2[1]))

	def render(self):
		pygame.draw.line(screen, self.color, self.start, self.p1, self.weight)
		prev_point = self.p1
		for point in zip(self.x, self.y):
			pygame.draw.line(screen, self.color, prev_point, point, self.weight)
			prev_point = point
		pygame.draw.line(screen, self.color, self.p2, self.end, self.weight)
	

def G(y,t): 
	x_d, θ_d, x, θ = y[0], y[1], y[2], y[3]

	x_dd = (l0+x) * θ_d**2 - k/m*x + g*cos(θ) # EOM
	θ_dd = -2.0/(l0+x) * x_d * θ_d - g/(l0+x) * sin(θ) # EOM

	return np.array([x_dd, θ_dd, x_d, θ_d])

def RK4_step(y, t, dt):
	k1 = G(y,t)
	k2 = G(y+0.5*k1*dt, t+0.5*dt)
	k3 = G(y+0.5*k2*dt, t+0.5*dt)
	k4 = G(y+k3*dt, t+dt)

	return dt * (k1 + 2*k2 + 2*k3 + k4) / 6

def update(x, θ):
	x_coord = scale*(l0+x) * sin(θ) + offset[0]
	y_coord = scale*(l0+x) * cos(θ) + offset[1]

	return (int(x_coord), int(y_coord))

def render(point):
	x, y = point[0], point[1]

	if prev_point:
		pygame.draw.line(trace, LT_BLUE, prev_point, (x, y), 5)

	screen.fill(WHITE)	
	if is_tracing:
		screen.blit(trace, (0,0))

	# pygame.draw.line(screen, BLACK, offset, (x,y), 5)
	s.update(offset, point)
	s.render()
	pygame.draw.circle(screen, BLACK, offset, 8)
	pygame.draw.circle(screen, BLUE, (x, y), int(m*10))

	return (x, y)

w, h = 1024, 768
WHITE = (255,255,255)
BLACK = (0,0,0)
RED = (255,0,0)
BLUE = (0,0,255)
LT_BLUE = (230,230,255)
offset = (w//2, h//4)
scale = 100
is_tracing = True

screen = pygame.display.set_mode((w,h))
screen.fill(WHITE)
trace = screen.copy()
pygame.display.update()
clock = pygame.time.Clock()

**Simulation**

In [4]:
pygame.font.init()
myfont = pygame.font.SysFont('Comic Sans MS', 38)

s = Spring(BLACK, (0,0), (0,0), 25, 30, 90, 90)

running = True
while running:
	for event in pygame.event.get():

		if event.type == pygame.QUIT:
			running = False
			sys.exit()

		if event.type == KEYDOWN:
			if event.key == K_t:
				is_tracing = not(is_tracing)
			if event.key == K_c:
				trace.fill(WHITE)
			if event.key == K_q:
				running = False
				# sys.exit()

	point = update(y[2], y[3])
	prev_point = render(point)

	time_string = 'Time: {} seconds'.format(round(t,1))
	text = myfont.render(time_string, False, (0, 0, 0))
	screen.blit(text, (10,10))

	t += delta_t
	y = y + RK4_step(y, t, delta_t) 

	clock.tick(60)
	pygame.display.update()

pygame.quit()